# Convert Raster Data to DGGS Zones

In [1]:
import os.path

import folium
import json
import pystac_client
import shapely
from tqdm.autonotebook import tqdm

from vgrid.conversion.raster2dggs import raster2a5, raster2rhealpix, raster2h3
from vgrid.utils.io import convert_to_output_format, download_file

/tmp/ipykernel_386257/1827741114.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# [optional] extra dependencies for nicer notebook progress bars if not available
!pip install -U tqdm ipywidgets jupyterlab_widgets

## Identify the Study Area of Interest

In [3]:
with open("../manitoba_study_area/inputs/manitoba_study_area.geojson") as f:
    data = f.read()
    poly = json.loads(data)
    geom = shapely.from_geojson(data)

map = folium.Map(location=[geom.centroid.y, geom.centroid.x], zoom_start=8)
folium.GeoJson(poly).add_to(map)
map

## Find RCM ARD Data from EODMS matching AOI

In [4]:
client = pystac_client.Client.open("https://www.eodms-sgdot.nrcan-rncan.gc.ca/stac/")
result = client.search(
    collections=["rcm-ard"],
    intersects=geom,
    datetime=["2018-01-01", "2026-01-01"],
)
result.matched()

74

In [5]:
item = next(result.items())
item

<Item id=9b27638f-6956-48e1-b66e-8f82df5cbbf5>

## Convert RCM ARD Data to DGGS

#### NOTE

⚠️ Requires patch: https://github.com/opengeoshub/vgrid/pull/57


In [ ]:
output_formats = ["geojson", "parquet"]
dggs_types_res = [
    ("H3", raster2h3, [7, 8]),  # H3 L8 hexagon dimension ~= ISEA7H L9
    # ("A5", raster2a5, [9]),
    # ("rHEALPix", raster2rhealpix, [8, 9]),
]

dggs_bar = tqdm(dggs_types_res, desc="Processing DGGS Types")
for dggrs, dggrs_builder, dggrs_resolutions in dggs_bar:
    dggs_bar.set_postfix(dggrs=dggrs)
    res_bar = tqdm(dggrs_resolutions, desc="Processing Resolutions", leave=False, total=len(dggrs_resolutions))
    for resolution in res_bar:
        res_bar.set_postfix(dggrs=dggrs, resolution=resolution)
        stac_items_bar = tqdm(result.items(), desc="Processing STAC Items", leave=False, total=result.matched())
        for item in stac_items_bar:
            stac_items_bar.set_postfix(dggrs=dggrs, resolution=resolution, item=item.id)
            date = item.properties["datetime"][:10]  # YYYY-MM-DD
            for band in ["rl", "rr"]:
                src_uri = item.assets[band].href
                src_name = os.path.basename(src_uri)
                res_bar.set_postfix(dggrs=dggrs, resolution=resolution, item=src_name)

                src_dir = os.path.join("./outputs", f"manitoba_rcm_ard", "src", date, band)
                src_path = os.path.join(src_dir, src_name)
                out_dir = os.path.join("./outputs", "manitoba_rcm_ard", dggrs, f"L{resolution}", date, band)
                os.makedirs(out_dir, exist_ok=True)
                out_files = {
                    fmt: os.path.join(out_dir, f"{os.path.splitext(src_name)[0]}.{fmt}")
                    for fmt in output_formats
                }
                if all(os.path.isfile(p) for p in out_files.values()):
                    print(f"  Skipping... All output files already exist: {list(out_files.values())}")
                    continue

                print(f"Converting to {dggrs}-L{resolution} [band={band}]. Saving to [{out_dir}]...")
                if not os.path.isfile(src_path):
                    os.makedirs(src_dir, exist_ok=True)
                    download_file(src_uri, src_path)
                out_df = dggrs_builder(
                    src_path,
                    resolution=resolution,
                    output_format="gpd"  # GeoPandas Dataframe
                )
                # save results
                for fmt, out_path in out_files.items():
                    convert_to_output_format(out_df, output_format=fmt, output_name=out_path)

Processing DGGS Types:   0%|          | 0/1 [00:00<?, ?it/s]

Processing Resolutions:   0%|          | 0/2 [00:00<?, ?it/s]

Processing STAC Items:   0%|          | 0/74 [00:00<?, ?it/s]

Converting to H3-L7 [band=rl]. Saving to [./outputs/manitoba_rcm_ard/H3/L7/2025-08-20/rl]...
